# Monads for Clojure Programmers

* Webpage: [You could have invented monads!](http://blog.sigfpe.com/2006/08/you-could-have-invented-monads-and.html?utm_source=pocket_mylist)
* Video: [Monads and Gonads](https://www.youtube.com/watch?v=b0EF0VTs9Dc)
* Video: [Railway oriented programming](https://vimeo.com/113707214) by Scott Wlaschin
    * Website: Scott Wlashin [F# For Fun and Profit](https://fsharpforfunandprofit.com/)

[Monads for Clojure Programmers](https://cuddly-octo-palm-tree.com/posts/2021-10-03-monads-clojure/)
* [What is a monad?](https://cuddly-octo-palm-tree.com/posts/2021-04-11-monads-0/)


## What makes a monad?

### Cheap version of core.match

In [2]:
(defmacro match
    [expr & cases]
    (let [e (gensym)]
        `(let [~e ~expr]
            (case (first ~e)
                ~@(->> (partition 2 cases)
                   (mapcat (fn [[pat body]]
                               [(first pat)
                                `(let [~(vec (cons '_ (rest pat))) ~e]
                                     ~body)])))))))

#'beaker_clojure_shell_340ae6bd-629c-4bb5-aa08-2f136ddd00f0/match

In [4]:
;; why doesn't this work????
(macroexpand-1 '(match expr
                    [:lit v] [v env]
                    [:var idx] [(get env idx) env]
                    [:set idx e] (let [[v env] (h e env)]
                                    [v (assoc env idx v)])))

java.util.ArrayList cannot be cast to java.util.Map: java.util.ArrayList cannot be cast to java.util.Map

### monadic do ... like Haskell's do operator

In [5]:
(defmacro mdo
    [bindings]
    (if (#{0 1} (count bindings))
       (throw (RuntimeException. "Invalid number of elements in mdo binding"))
       (let [[n v & r] bindings]
           (if (empty? r)
                v
                [:bind v `(fn [~n] (mdo ~r))]))))

#'beaker_clojure_shell_340ae6bd-629c-4bb5-aa08-2f136ddd00f0/mdo

In [6]:
(macroexpand-1 '(mdo [
                    a [:return 5]
                    b [:return 3]
                    c [:return (+ a b)]
                    _ [:set 0 c]
                    v [:get 2]
                    _ [:return (+ v c)]
]))

java.util.ArrayList cannot be cast to java.util.Map: java.util.ArrayList cannot be cast to java.util.Map

## First example, ambient state

In [8]:
(defn run-ambient
    ([m] (run-ambient m {}))
    ([m env]
        (match m
            [:return v] [v env]
            [:bind ma f] (let [[v env] (run-ambient ma env)]
                            (run-ambient (f v) env))
            [:set k v] [v (assoc env k v)]
            [:get k] [(get env k) env]
            [:update k f] (let [v (f (get env k))]
                            [v (assoc env k v)]))))

#'beaker_clojure_shell_340ae6bd-629c-4bb5-aa08-2f136ddd00f0/run-ambient

In [10]:
(run-ambient
    (mdo [
            a [:return 5]
            b [:return 3]
            c [:return (+ a b)]
            _ [:set 0 c]
            v [:get 2]
            _ [:return (+ v c)]
    ])
    {2 3}
    )

[11, {2=3, 0=8}]

### Expression counter

In [12]:
(defn sequenceM
    "Takes a list of monadic values ms, and returns
    a single monadic value that wraps a list"
    [ms] ;; sequence of monadic values
    (if (empty? ms)
        [:return ()]
        (mdo [
            a (first ms)
            r (sequenceM (rest ms))
            _ [:return (cons a r)]
            ]))
    )

#'beaker_clojure_shell_340ae6bd-629c-4bb5-aa08-2f136ddd00f0/sequenceM

In [14]:
(defn count-exprs
    [expr]
    (let [m-inc (fn [k] [:update k (fnil inc 0)])]
        (match expr
            [:lit e] (mdo [_ (m-inc :lit)
                           _ [:return 1]])
            [:var _] (mdo [_ (m-inc :var)
                           _ [:return 1]])
            [:set _ e] (mdo [
                              c (count-exprs e)
                              _ (m-inc :set)
                              _ [:return (inc c)]
                            ])
            [:bin _ e1 e2] (mdo [
                                  c1 (count-exprs e1)
                                  c2 (count-exprs e2)
                                  _ (m-inc :bin)
                                  _ [:return (+ c1 c2 1)]
                                ])
            [:while e-cond e-body] (mdo [
                                          c1 (count-exprs e-cond)
                                          c2 (count-exprs e-body)
                                          _ (m-inc :while)
                                          _ [:return (+ c1 c2 1)]
                                        ])
            [:do & exprs] (mdo [
                                 counts (sequenceM (mapv count-exprs exprs))
                                 _ (m-inc :do)
                                 _ [:return (reduce + 1 counts)]
            ]))))

#'beaker_clojure_shell_340ae6bd-629c-4bb5-aa08-2f136ddd00f0/count-exprs

In [16]:
(run-ambient 
    (count-exprs
      [:do
         [:set 0 [:lit 100]]
         [:set 1 [:lit 1000]]
         [:while
            [:bin :not= [:lit 0] [:var 1]]
            [:do
                [:set 0 [:bin :add 
                              [:bin :add 
                                    [:bin :add 
                                          [:var 0] 
                                          [:lit 4]]
                                    [:var 0]]
                              [:lit 3]]]
                [:set 0 [:bin :add
                              [:bin :add [:var 0] [:lit 2]]
                              [:lit 4]]]
                [:set 1 [:bin :add [:lit -1] [:var 1]]]]]
         [:var 0]]))

[29, {:lit=8, :set=5, :var=6, :bin=7, :do=2, :while=1}]

## Second example: non-deterministic computation

In [18]:
(defn run-nd
    ([ma] (run-nd ma []))
    ([ma s]
        (match ma
            [:return a] [a]
            [:bind ma f] (mapcat (comp run-nd f) (run-nd ma))
            [:multi ls] ls)))

#'beaker_clojure_shell_340ae6bd-629c-4bb5-aa08-2f136ddd00f0/run-nd

In [21]:
(run-nd (mdo [
               a [:multi [1 2]]
               b [:multi [3 4]]
               _ [:return (* a b)]
]))

[3, 4, 6, 8]

In [22]:
(defn filter-pos
   [a]
   [:multi (if (pos? a) [a] [])])

#'beaker_clojure_shell_340ae6bd-629c-4bb5-aa08-2f136ddd00f0/filter-pos

In [23]:
(run-nd (mdo [
                a [:multi [-1 2]]
                b [:multi [3 4]]
                c [:return (* a b)]
                d (filter-pos c)
                _ [:return d]
]))

[6, 8]

### Sovle quadratic equations

In [25]:
(defn sqrt
    [x]
    (cond (neg? x) [:multi []]
          (zero? x) [:return 0]
          (pos? x) (mdo [
                          sign [:multi [-1 1]]
                          _    [:return (* sign (Math/sqrt x))]
          ])))
          
(defn div
    [a b]
    (if (zero? b)
        [:multi []]
        [:return (/ a b)]))
        
(defn solve-2nd
    [a b c]
    (mdo [
            d (sqrt (- (* b b) (* 4 a c)))
            x (div (- d b) (* 2 a))
    ]))

#'beaker_clojure_shell_340ae6bd-629c-4bb5-aa08-2f136ddd00f0/solve-2nd

In [26]:
(run-nd (solve-2nd 1 2 1))

[-1]

In [27]:
(run-nd (solve-2nd 1 3 1))

[-2.618033988749895, -0.3819660112501051]

In [28]:
(run-nd (solve-2nd -2 0 -1))

[]

In [29]:
(run-nd (solve-2nd 1 -2 1))

[1]